In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import markdownify
import re
from datasets import Dataset
from tqdm import tqdm
import itertools

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [82]:
# url = "https://essayforum.com/writing/average-cell-phone-national-fixed-87878/"
# url = "https://essayforum.com/writing/people-think-students-study-science-food-94982/"
base_url = (
    "https://essayforum.com/writing/shortage-housing-big-cities-cause-severe-94815/"
)
response = requests.get(url=base_url)
soup = BeautifulSoup(response.text)
all_messages = soup.find_all(
    "article",
)

In [ ]:
import pprint

pprint.pprint(str(all_messages[0]))

In [59]:
author = all_messages[0].find("div", "fl").find_all("a")
if len(author) > 0:
    author = author[0].find("b").text
else:
    author = all_messages[0].find("div", "fl").text.replace("/", "").strip()
author

'Hazard'

In [37]:
number = all_messages[0].find("div", "fr").find("a").text
date = all_messages[0].find("div", "fr").text.replace(number, "").strip()
date

'3 hrs ago'

In [ ]:
message = str(all_messages[0].find("div", "pTx"))

message = markdownify.markdownify(message, heading_style="ATX")
message = re.sub("\!\[\]\(\w+.*\)", "", message).strip()
print(message)

In [ ]:
for message in all_messages:
    author = message.find("div", "fl").find("a").find("b").text.strip()
    print(author)

    number = all_messages[0].find("div", "fr").find("a").text
    date = all_messages[0].find("div", "fr").text.replace(number, "").strip()
    print(date)

    message = str(message.find("div", "pTx"))
    message = markdownify.markdownify(message, heading_style="ATX")
    message = re.sub("\!\[\]\(\w+.*\)", "", message).strip()
    print(message)
    print("=" * 100)
    print("=" * 100)

In [2]:
from verbalist.datasets.essayforum.essayforum import EssayForumMessagesParser


links = pd.read_csv("./verbalist/datasets/essayforum/essayforum_writing_links.csv")
links = links.to_dict("records")
links = links[:10]

parser = EssayForumMessagesParser(urls=links)
parser.parse()

  0%|          | 0/10 [00:00<?, ?it/s]

/cephfs/home/kosenko/verbalist/verbalist/datasets/essayforum/essayforum.py:26: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 26 of the file /cephfs/home/kosenko/verbalist/verbalist/datasets/essayforum/essayforum.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(response.text)
100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


In [3]:
Dataset.from_list(parser.dataset)

Dataset({
    features: ['message', 'author', 'date', 'position', 'url', 'forum_type'],
    num_rows: 13
})

In [ ]:
parser.dataset

### parse links


In [64]:
base_url = "https://essayforum.com/writing/2/"
response = requests.get(url=base_url)
soup = BeautifulSoup(response.text)
all_links = soup.find_all("a", class_="txtNr")

In [70]:
all_links[0]["href"]

'https://essayforum.com/writing/discuss-view-give-opinion-zoo-closed-due-94830/'

In [68]:
total_pages = soup.find_all("a", class_="navCell")[-1].text
total_pages = int(total_pages)

'478'

In [73]:
from verbalist.datasets.essayforum.essayforum import EssayForumLinksParser

links_parser = EssayForumLinksParser(urls=["https://essayforum.com/writing"])
links_parser.parse()

100%|██████████| 478/478 [05:56<00:00,  1.34it/s]


In [75]:
len(links_parser.dataset)

28669

In [77]:
pd.DataFrame(links_parser.dataset).to_csv(
    "./verbalist/datasets/essayforum/essayforum_writing_links.csv",
    index=False,
)

### convert into prompts for llm


In [31]:
from datasets import load_dataset, Dataset

dataset = load_dataset("dim/essayforum_writing_10k")
dataset = dataset["train"].to_pandas()
dataset.head(5)

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--essayforum_writing_10k-208b845e5ffb772e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 568.26it/s]


,message,author,date,position,url,forum_type
0,## Some companies block their employees from u...,mrhuyprono1,"Feb 4, 2023",0,https://essayforum.com/writing/toeic-part-comp...,essayforum.com_writing_
1,The presentation is not grammatically perfect ...,Holt,"Feb 4, 2023",1,https://essayforum.com/writing/toeic-part-comp...,essayforum.com_writing_
2,***Some people believe that the personal infor...,Heo Nguyen,"Jan 28, 2023",0,https://essayforum.com/writing/violent-crimina...,essayforum.com_writing_
3,"It is a common belief \nA ""common belief"" can...",Holt,"Jan 28, 2023",1,https://essayforum.com/writing/violent-crimina...,essayforum.com_writing_
4,"Regarding the introduction, it is my view that...",Camtu25,"Jan 28, 2023",2,https://essayforum.com/writing/violent-crimina...,essayforum.com_writing_


In [32]:
write_reviews = {}

for i in range(len(dataset)):
    url = dataset["url"].iloc[i]
    pos = dataset["position"].iloc[i]
    message = dataset["message"].iloc[i]
    author = dataset["author"].iloc[i]
    if url in write_reviews and author == "Holt":
        write_reviews[url]["answer"] = message
    elif not url in write_reviews:
        write_reviews[url] = {}
        write_reviews[url]["prompt"] = message

write_reviews = list(write_reviews.values())
write_reviews = [item for item in write_reviews if "answer" in item]
# write_reviews

In [34]:
Dataset.from_list(write_reviews).to_pandas().to_csv(
    "./verbalist/datasets/essayforum/essayforum_writing_prompts_6k.csv",
    index=False,
)

In [36]:
Dataset.from_list(write_reviews).push_to_hub("dim/essayforum_writing_prompts_6k")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:04<00:00,  4.21s/it]


In [ ]:
Dataset.from_list(write_reviews).to_pandas()

In [1]:
from datasets import load_dataset

dataset = load_dataset("dim/essayforum_writing_prompts_6k")
dataset["train"][0]

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 250.32it/s]


Dataset parquet downloaded and prepared to /home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--essayforum_writing_prompts_6k-a911bbef273f1337/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 643.20it/s]


{'prompt': "## Some companies block their employees from using social media network\n\n  \n  \nIn many corporations, despite its rules, staff cannot access social media platforms like Facebook, Twitter, and Instagram during working time. Personally, I totally agree with that decision. Indeed, prohibiting employees from using social media can help them avoid getting distracted, get more attention to their tasks, build real-life relationships with other employees, and lead to a result that boosts company performance.  \n  \nWithout a doubt, social media is a common source of interruption. For example, people can quickly get interested in something new that pops up in their newsfeed or their friend's messages when using Facebook. Some researchers said that you need an average of 20 minutes to get back to work when visiting social media platforms.  \n  \nSo that, if they are prohibited from using social media, employees will hardly get any other distraction sources in the office. For that 

### generate prompts with tasks, chatgpt


In [2]:
import openai
import pandas as pd
import numpy as np
import time

openai.api_key = open("./chat_gpt_token").read()

In [12]:
def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    chat_completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    return chat_completion["choices"][0]["message"]["content"].strip()


user_prompt = "Hello world."
chatbot_response = chat_with_chatgpt(user_prompt)
print(chatbot_response)

Hello! How can I assist you today?


In [ ]:
from datasets import load_dataset

dataset = load_dataset("dim/essayforum_writing_prompts_6k")
dataset = dataset["train"]

In [9]:
dataset[0]

{'prompt': "## Some companies block their employees from using social media network\n\n  \n  \nIn many corporations, despite its rules, staff cannot access social media platforms like Facebook, Twitter, and Instagram during working time. Personally, I totally agree with that decision. Indeed, prohibiting employees from using social media can help them avoid getting distracted, get more attention to their tasks, build real-life relationships with other employees, and lead to a result that boosts company performance.  \n  \nWithout a doubt, social media is a common source of interruption. For example, people can quickly get interested in something new that pops up in their newsfeed or their friend's messages when using Facebook. Some researchers said that you need an average of 20 minutes to get back to work when visiting social media platforms.  \n  \nSo that, if they are prohibited from using social media, employees will hardly get any other distraction sources in the office. For that 

In [23]:
num = 80
question = dataset[num]["prompt"]
solution = dataset[num]["answer"]

input_text = f"""
Essay: {question}
Evaluation: {solution}

write a prompt based on 'Essay' and 'Evaluation' that asks to evaluate the essay. Answer only prompt.
"""
print(input_text)
chatbot_response = chat_with_chatgpt(input_text)
print("=" * 100)
print("=" * 100)
print(chatbot_response)


Essay: ## nitrogen oxide emission

  
  
The given line chart provided comparison of the amount of nitrogen oxide released by diesel car, petrol car, lorries and buses. From an overall perspective, it is evident that buses and lorries are more pollutant than two remained cars. While diesel car and petrol car dispose the same amount of emission as the speed increase, the figure for buses and lorries fluctuate mildly.  
  
At the slowest level of pace, buses account for the highest amount of nitrogen oxide gas produced, at precisely 40 gam per kilometer. By contrast, lorries release nearly 33 gam per kilometer, which is slightly lower than buses but still much higher than other vehicles. The figure for diesel car and petrol car stands at approximately 7 g/k and 12g/k, respectively.  
  
With the speed increases from 10 to 130 kph, the nitrogen oxide waste of buses experiences an oscillation, with the amount of emission decreases to 22 g/k at 60 kph before goes up to 45 g/k at the maximu

Prompt: On a scale of 1-10, rate the essay based on its clarity, organization, and use of evidence to support the argument. Justify your rating with specific examples from the essay and evaluation.


In [28]:
evaluation_prompts = [
    "rate the essay based on its clarity, organization, and use of evidence to support the argument. Justify your rating with specific examples from the essay and evaluation.",
    "Rate the essay based on the clarity of the writer's opinion, development of reasoning paragraphs, and effectiveness of the concluding summary.",
    "Rate the essay based on the writer's ability to present a clear and cohesive argument, provide relevant examples and reasoning, and effectively address the topic.",
    "Evaluate the essay provided in terms of coherence, cohesiveness, and the appropriateness of the response to the given question.",
    "Evaluate the essay by discussing its strengths and weaknesses.",
    "Evaluate the essay provided above. Assess the writer's ability to follow the given instructions and provide a well-structured and coherent response.",
    "Evaluate the essay based on the given prompt and provide your analysis.",
    "Please rate the essay based on how clear it is, how well it is organized, and how effectively it uses evidence to support the argument. Please justify your rating with specific examples from the essay and your evaluation.",
    "Hey there! Can you please rate this essay for me? I'd like you to consider how clear it is, how well it's organized, and how effectively it uses evidence to support its argument. It would be great if you could back up your rating with specific examples from the essay and explain your evaluation. Thanks a bunch!",
    "Rate the essay based on how clear the writer's opinion is, how well they develop their reasoning paragraphs, and how effective their concluding summary is.",
    "Rate the essay based on the writer's ability to present a clear and cohesive argument, provide relevant examples and reasoning, and effectively address the topic. Can you do that for me, please?",
    "Hey there! Can you please take a look at this essay and tell me what you think about its coherence, cohesiveness, and how well it responds to the given question? Thanks!",
    "Hey there! Can you take a look at this essay and give me your thoughts on it? I'd love to hear about its strengths and weaknesses.",
    "Can you please evaluate the essay above? I want to know if the writer followed the instructions properly and if their response is well-structured and coherent.",
    "hey, can you read this essay and tell me what you think?",
]

In [27]:
for prompt in evaluation_prompts:
    # prompt = evaluation_prompts[0]
    input_text = f"""
	Prompt: {prompt}

	Make this prompt more human like. You can add small letters, errors and everything you want.
	"""
    print(prompt)
    print("=" * 100)
    chatbot_response = chat_with_chatgpt(input_text)
    print(chatbot_response)
    print("=" * 100)
    print("=" * 100)

rate the essay based on its clarity, organization, and use of evidence to support the argument. Justify your rating with specific examples from the essay and evaluation.
Hey there! Can you please rate this essay for me? I'd like you to consider how clear it is, how well it's organized, and how effectively it uses evidence to support its argument. It would be great if you could back up your rating with specific examples from the essay and explain your evaluation. Thanks a bunch!
Rate the essay based on the clarity of the writer's opinion, development of reasoning paragraphs, and effectiveness of the concluding summary.
Rate the essay based on how clear the writer's opinion is, how well they develop their reasoning paragraphs, and how effective their concluding summary is.
Rate the essay based on the writer's ability to present a clear and cohesive argument, provide relevant examples and reasoning, and effectively address the topic.
Rate the essay based on the writer's ability to present

In [30]:
summarized_prompts = []
for prompt in evaluation_prompts:
    # prompt = evaluation_prompts[0]
    input_text = f"""
	Prompt: {prompt}

	Summarize this prompt in 4-8 words.
	"""
    # print(prompt)
    # print("="*100)
    chatbot_response = chat_with_chatgpt(input_text)
    summarized_prompts.append(chatbot_response)
    # print(chatbot_response)
    # print("="*100)
    # print("="*100)

In [31]:
summarized_prompts

['Rate essay on clarity, organization, and evidence.',
 'Rate essay on clarity, development, and conclusion.',
 'Rate essay on clarity, examples, reasoning, and topic.',
 'Evaluate coherence, cohesiveness, and appropriateness of essay.',
 "Evaluate essay's strengths and weaknesses.",
 'Evaluate essay structure and coherence.',
 'Evaluate and analyze an essay based on a prompt.',
 'Rating essay clarity, organization, and evidence usage.',
 'Rate essay on clarity, organization, and evidence.',
 'Rate essay on clarity, reasoning development, and conclusion.',
 'Rate essay on clarity, examples, reasoning, and addressing topic.',
 'Prompt summary: Evaluate essay coherence, cohesiveness, and relevance.',
 "Evaluate essay's strengths and weaknesses.",
 'Evaluate essay structure and coherence.',
 'Request for feedback on an essay.']

In [33]:
evaluation_prompts + summarized_prompts

['rate the essay based on its clarity, organization, and use of evidence to support the argument. Justify your rating with specific examples from the essay and evaluation.',
 "Rate the essay based on the clarity of the writer's opinion, development of reasoning paragraphs, and effectiveness of the concluding summary.",
 "Rate the essay based on the writer's ability to present a clear and cohesive argument, provide relevant examples and reasoning, and effectively address the topic.",
 'Evaluate the essay provided in terms of coherence, cohesiveness, and the appropriateness of the response to the given question.',
 'Evaluate the essay by discussing its strengths and weaknesses.',
 "Evaluate the essay provided above. Assess the writer's ability to follow the given instructions and provide a well-structured and coherent response.",
 'Evaluate the essay based on the given prompt and provide your analysis.',
 'Please rate the essay based on how clear it is, how well it is organized, and how 

In [ ]:
selected_prompts = [
    "rate the essay based on its clarity, organization, and use of evidence to support the argument. Justify your rating with specific examples from the essay and evaluation.",
    "Rate the essay based on the clarity of the writer's opinion, development of reasoning paragraphs, and effectiveness of the concluding summary.",
    "Rate the essay based on the writer's ability to present a clear and cohesive argument, provide relevant examples and reasoning, and effectively address the topic.",
    "Evaluate the essay provided in terms of coherence, cohesiveness, and the appropriateness of the response to the given question.",
    "Evaluate the essay by discussing its strengths and weaknesses.",
    "Evaluate the essay provided above. Assess the writer's ability to follow the given instructions and provide a well-structured and coherent response.",
    "Evaluate the essay based on the given prompt and provide your analysis.",
    "Please rate the essay based on how clear it is, how well it is organized, and how effectively it uses evidence to support the argument. Please justify your rating with specific examples from the essay and your evaluation.",
    "Hey there! Can you please rate this essay for me? I'd like you to consider how clear it is, how well it's organized, and how effectively it uses evidence to support its argument. It would be great if you could back up your rating with specific examples from the essay and explain your evaluation. Thanks a bunch!",
    "Rate the essay based on how clear the writer's opinion is, how well they develop their reasoning paragraphs, and how effective their concluding summary is.",
    "Rate the essay based on the writer's ability to present a clear and cohesive argument, provide relevant examples and reasoning, and effectively address the topic. Can you do that for me, please?",
    "Hey there! Can you please take a look at this essay and tell me what you think about its coherence, cohesiveness, and how well it responds to the given question? Thanks!",
    "Hey there! Can you take a look at this essay and give me your thoughts on it? I'd love to hear about its strengths and weaknesses.",
    "Can you please evaluate the essay above? I want to know if the writer followed the instructions properly and if their response is well-structured and coherent.",
    "hey, can you read this essay and tell me what you think?",
    "Rate essay on clarity, organization, and evidence.",
    "Rate essay on clarity, development, and conclusion.",
    "Rate essay on clarity, examples, reasoning, and topic.",
    "Evaluate coherence, cohesiveness, and appropriateness of essay.",
    "Evaluate essay's strengths and weaknesses.",
    "Evaluate essay structure and coherence.",
    "Evaluate and analyze an essay based on a prompt.",
    "Rating essay clarity, organization, and evidence usage.",
    "Rate essay on clarity, organization, and evidence.",
    "Rate essay on clarity, reasoning development, and conclusion.",
    "Rate essay on clarity, examples, reasoning, and addressing topic.",
    "Prompt summary: Evaluate essay coherence, cohesiveness, and relevance.",
    "Evaluate essay's strengths and weaknesses.",
    "Evaluate essay structure and coherence.",
    "Request for feedback on an essay.",
    'evaluate this essay',
    'evaluate essay',
    'rate essay',
]

